In [1]:
from aocd import submit, get_data
from typing import List
from functools import reduce
import itertools
import numpy as np
import pandas as pd
import re
from collections import Counter
import networkx as nx


def str2intList(s: str) -> List[int]:  # str2intList("2\n3") -> [2,3]
    return [int(x) for x in s.strip().split("\n")]

def imag2tup(i: complex) -> tuple: # 1+2j -> (1, 2)
    return (int(i.real), int(i.imag))

def tup2imag(t: tuple) -> complex:
    return t[0] + t[1] * 1j

def generator(method):
    def annotation(day, quiet=True, printdata=False):
        def solver(solve):
            part = 'ab'[method]
            if(data[day]):
                esol = solve(data[day], method=method)
                print(f"Ex. {day}{part}", esol, sep="\t")
            sol = solve(get_data(day=day), method=method)
            print(f"Lvl {day}{part}", sol, sep="\t")
            submit(sol, part, day=day, quiet=quiet)
            return solve # return the original function, so you can put another annotation on it
        return solver # 
    return annotation

level_a, level_b = generator(0), generator(1)

In [2]:
data = ['',
 '1721\n979\n366\n299\n675\n1456',
 '1-3 a: abcde\n1-3 b: cdefg\n2-9 c: ccccccccc',
 '..##.......\n#...#...#..\n.#....#..#.\n..#.#...#.#\n.#...##..#.\n..#.##.....\n.#.#.#....#\n.#........#\n#.##...#...\n#...##....#\n.#..#...#.#',
 'ecl:gry pid:860033327 eyr:2020 hcl:#fffffd\nbyr:1937 iyr:2017 cid:147 hgt:183cm\n\niyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884\nhcl:#cfa07d byr:1929\n\nhcl:#ae17e1 iyr:2013\neyr:2024\necl:brn pid:760753108 byr:1931\nhgt:179cm\n\nhcl:#cfa07d eyr:2025 pid:166559648\niyr:2011 ecl:brn hgt:59in\neyr:1972 cid:100\nhcl:#18171d ecl:amb hgt:170 pid:186cm iyr:2018 byr:1926\n\niyr:2019\nhcl:#602927 eyr:1967 hgt:170cm\necl:grn pid:012533040 byr:1946\n\nhcl:dab227 iyr:2012\necl:brn hgt:182cm pid:021572410 eyr:2020 byr:1992 cid:277\n\nhgt:59cm ecl:zzz\neyr:2038 hcl:74454a iyr:2023\npid:3556412378 byr:2007\npid:087499704 hgt:74in ecl:grn iyr:2012 eyr:2030 byr:1980\nhcl:#623a2f\n\neyr:2029 ecl:blu cid:129 byr:1989\niyr:2014 pid:896056539 hcl:#a97842 hgt:165cm\n\nhcl:#888785\nhgt:164cm byr:2001 iyr:2015 cid:88\npid:545766238 ecl:hzl\neyr:2022\n\niyr:2010 hgt:158cm hcl:#b6652a ecl:blu byr:1944 eyr:2021 pid:093154719',
 'FBFBBFFRLR',
 'abc\n\na\nb\nc\n\nab\nac\n\na\na\na\na\n\nb',
 'light red bags contain 1 bright white bag, 2 muted yellow bags.\ndark orange bags contain 3 bright white bags, 4 muted yellow bags.\nbright white bags contain 1 shiny gold bag.\nmuted yellow bags contain 2 shiny gold bags, 9 faded blue bags.\nshiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.\ndark olive bags contain 3 faded blue bags, 4 dotted black bags.\nvibrant plum bags contain 5 faded blue bags, 6 dotted black bags.\nfaded blue bags contain no other bags.\ndotted black bags contain no other bags.'
 '', '', '', ''];

In [3]:
@level_b(1)
@level_a(1)
def solve(data, method=0):
    data = sorted(str2intList(data))
    for line in itertools.product(data, repeat=method+2):
        if np.sum(line) == 2020:
            return np.prod(line)

Ex. 1a	514579
Lvl 1a	996996
Ex. 1b	241861950
Lvl 1b	9210402


In [4]:
@level_b(2)
@level_a(2)
def solve(data, method):
    cnt = 0
    for line in data.split("\n"):
        nr, letter, pw = line.split(" ")
        letter = letter[0]
        start, end = map(int, nr.split("-"))
        if method == 0:
            cnt += pw.count(letter[0]) in range(int(start), int(end)+1)
        else:
            cnt += (pw[start-1]+pw[end-1]).count(letter) == 1
    return cnt

Ex. 2a	2
Lvl 2a	519
Ex. 2b	1
Lvl 2b	708


In [5]:
@level_b(3)
@level_a(3)
def solve(data, method=0):
    def sol(data, mov):
        d = np.array(list(map(list, data.split("\n")))) == '#'
        pos = cnt = x = 0
        while True:
            pos += mov
            x, y = imag2tup(pos)
            if x >= d.shape[0]:
                return cnt
            cnt += d[x, y % d.shape[1]]
    
    if method:
        slopes = [1+1j, 1+3j, 1+5j, 1+7j, 2+1j]
        return np.prod([sol(data, mov=mov) for mov in slopes])
    else:
        return sol(data, mov=1+3j)

Ex. 3a	7
Lvl 3a	214
Ex. 3b	336
Lvl 3b	8336352024


In [6]:
@level_b(4)
@level_a(4)
def solve(data, method=0):
    ctr = 0
    ALLOWED_DICT = {
    "byr": lambda a: int(a) in range(1920, 2003), # (Birth Year) - four digits; at least 1920 and at most 2002.
    "iyr": lambda a: int(a) in range(2010, 2021), # (Issue Year) - four digits; at least 2010 and at most 2020.
    "eyr": lambda a: int(a) in range(2020, 2031), # (Expiration Year) - four digits; at least 2020 and at most 2030.
    "hgt": lambda a: (len(a) > 2) and (a[-2:] == "cm" and int(a[:-2]) in range(150, 194)) 
        or (a[-2:] == "in" and int(a[:-2]) in range(59, 77)), #(Height) - a number followed by either cm or in:        If cm, the number must be at least 150 and at most 193.        If in, the number must be at least 59 and at most 76.
    "hcl": lambda a: len(a) == 7 and a[0] == "#" and set(a[1:]).issubset(set("0123456789abcdef")), # (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
    "ecl": lambda a: a in ["amb", "blu", "brn", "gry", "grn", "hzl", "oth"], # (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
    "pid": lambda a: len(a) == 9 and set(a).issubset(set("0123456789")), # (Passport ID) - a nine-digit number, including leading zeroes.
    }
    for line in data.split("\n\n"):
        ut = {}
        for token in re.split("[ \n]+", line):
            k, v = token.split(":")
            ut[k] = v
        if method: # part b
            stats = [k in ut and v(ut[k]) for k, v in ALLOWED_DICT.items()]
            ctr += all(stats)
        else: # part a
            ctr += set(ut).issubset(set(ALLOWED_DICT))
    return ctr

Ex. 4a	4
Lvl 4a	125
Ex. 4b	6
Lvl 4b	116


In [9]:
@level_b(5)
@level_a(5)
def solve(data, method):
    data = data.replace("F", "0").replace("B", "1").replace("L", "0").replace("R", "1")
    data = [int(d, 2) for d in data.split("\n")]
    if method:
        if len(data) > 1:
            return (set(range(min(data), max(data))) - set(data)).pop()
    else:
        return max(data)

Ex. 5a	357
Lvl 5a	801
Ex. 5b	None
Lvl 5b	597


In [10]:
@level_b(6)
@level_a(6)
def solve(data, method=0):
    cnt = 0
    for section in data.split("\n\n"):
        nr_lines = section.count("\n") + 1
        if method==0:
            cnt += len(set(section).difference({"\n"}))
        else:
            maxcnt = section.count("\n") + 1
            cnt += sum([lcnt == maxcnt and letter != "\n" for letter, lcnt in Counter(section).items()])
    return cnt

Ex. 6a	11
Lvl 6a	6416
Ex. 6b	6
Lvl 6b	3050


In [11]:
@level_b(7)
@level_a(7)
def solve(data, method=0):
    def dfs(G, start, factor):
        return sum([dfs(G, follower, factor * val["cnt"]) + factor * val["cnt"] 
                    for follower, val in G[start].items()])

    G = nx.DiGraph()
    for line in data.split("\n"):
        line = re.sub("(bags?)", "", line)
        containing, contained = line[:-2].split("  contain ")
        if not "no other" in contained:
            for contain in contained.split(" , "):
                cspl = contain.split(" ")
                cnt = int(cspl[0]) if len(cspl) == 3 else 1
                contain = " ".join(cspl[-2:])
                G.add_edge(containing, contain, cnt=cnt)
    if method:
        return dfs(G, "shiny gold", 1)
    else:
        return len(nx.algorithms.dag.ancestors(G, "shiny gold"))        

Ex. 7a	4
Lvl 7a	252
Ex. 7b	32
Lvl 7b	35487


In [ ]:
data[8] = """"""

@level_b(8)
@level_a(8)
def solve(data, method=0):
    def dfs(G, start, factor):
        return sum([dfs(G, follower, factor * val["cnt"]) + factor * val["cnt"] 
                    for follower, val in G[start].items()])

    G = nx.DiGraph()
    for line in data.split("\n"):
        line = re.sub("(bags?)", "", line)
        containing, contained = line[:-2].split("  contain ")
        if not "no other" in contained:
            for contain in contained.split(" , "):
                cspl = contain.split(" ")
                cnt = int(cspl[0]) if len(cspl) == 3 else 1
                contain = " ".join(cspl[-2:])
                G.add_edge(containing, contain, cnt=cnt)
    if method:
        return dfs(G, "shiny gold", 1)
    else:
        return len(nx.algorithms.dag.ancestors(G, "shiny gold"))        